In [1]:
import json
import pandas as pd
from io import StringIO

SENTINEL = -999999999999.0

def hydrocron_response_to_df(response_text: str) -> pd.DataFrame:
    """
    Hydrocron sometimes returns JSON with CSV embedded at results.csv even when output=csv.
    This function returns a DataFrame in either case.
    """
    text = response_text.strip()

    # Case A: JSON wrapper with embedded CSV
    if text.startswith("{"):
        obj = json.loads(text)
        csv_text = obj.get("results", {}).get("csv", "")
        if not csv_text:
            return pd.DataFrame()
        return pd.read_csv(StringIO(csv_text))

    # Case B: raw CSV
    return pd.read_csv(StringIO(text))


In [2]:
import requests
from datetime import datetime, timedelta, timezone

HYDROCRON_URL = "https://soto.podaac.earthdatacloud.nasa.gov/hydrocron/v1/timeseries"

now = datetime.now(timezone.utc)
start = (now - timedelta(days=365)).strftime("%Y-%m-%dT%H:%M:%SZ")
end   = now.strftime("%Y-%m-%dT%H:%M:%SZ")

params = {
    "feature": "Reach",
    "feature_id": "22511300103",
    "start_time": start,
    "end_time": end,
    "output": "csv",
    "collection_name": "SWOT_L2_HR_RiverSP_2.0",
    "fields": "reach_id,time_str,cycle_id,pass_id,wse,slope,width,dschg_gm,dschg_gm_q,reach_q",
}

r = requests.get(HYDROCRON_URL, params=params, timeout=60)
print("Status:", r.status_code)

df = hydrocron_response_to_df(r.text)
print(df.head())
print("Rows:", len(df))


Status: 200
      reach_id              time_str  cycle_id  pass_id           wse  \
0  22511300103  2025-01-22T01:04:45Z        27      234 -7.850000e-02   
1  22511300103               no_data        27      333 -1.000000e+12   
2  22511300103  2025-02-01T23:27:32Z        27      540  2.070000e-02   
3  22511300103  2025-02-04T12:47:50Z        28       27  1.035000e-01   
4  22511300103  2025-02-11T21:49:48Z        28      234  2.010000e-01   

          slope         width      dschg_gm  dschg_gm_q  reach_q wse_units  \
0  4.789320e-06  9.289239e+02 -1.000000e+12        -999        1         m   
1 -1.000000e+12 -1.000000e+12 -1.000000e+12        -999        3         m   
2  2.074499e-05  1.114716e+03 -1.000000e+12        -999        1         m   
3  1.323430e-05  1.361866e+03 -1.000000e+12        -999        1         m   
4  2.594196e-05  9.131528e+02 -1.000000e+12        -999        1         m   

  slope_units width_units dschg_gm_units  
0         m/m           m          m^